In [19]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [2]:
df = pd.read_csv("../data/trialPromoResults.csv")
sex_map = {"M": 0, "F": 1}
mstatus_map = {"single":0, "married":1, "widowed":2, "divorced":3}
occupation_map = {'legal':0, 'IT':1, 'government':2, 'manuf':3, 'retired':4, 
                  'finance':5,'construct':6, 'education':7, 'medicine':8}
education_map = {'postgrad':3, 'secondary':0, 'tertiary':1, 'professional':2}
decision_map = {"A":0, "B":1, "None":2}
df["sex"] = df["sex"].map(sex_map)
df["mstatus"] = df["mstatus"].map(mstatus_map)
df["occupation"] = df["occupation"].map(occupation_map)
df["education"] = df["education"].map(education_map)
df["decision"] = df["decision"].map(decision_map)
df = pd.get_dummies(df, columns=["sex","mstatus","occupation","education"])

In [3]:
X = df[list(set(df.columns) - set(["index", "decision"]))]
y = df["decision"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)
normalizer = preprocessing.Normalizer()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

In [4]:
X_resampled, y_resampled = RandomOverSampler(ratio={0:200, 1:200}).fit_sample(X_train, y_train)
# X_resampled, y_resampled = RandomUnderSampler(ratio={2:250}).fit_sample(X_resampled, y_resampled)
X_train = X_resampled
y_train = y_resampled
print(sorted(Counter(y_train).items()))

[(0, 200), (1, 200), (2, 580)]


In [5]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(y_train)

Using TensorFlow backend.


In [20]:
def precision_metrics(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import metrics as kmetrics
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision_metrics])
model.fit(X_train, dummy_y, epochs=150, batch_size=20, class_weight="auto")

Epoch 1/150
980/980 [==============================] - 1s 795us/step - loss: 0.6028 - precision_metrics: 0.0000e+00
Epoch 2/150
980/980 [==============================] - 0s 406us/step - loss: 0.5760 - precision_metrics: 0.2828
Epoch 3/150
980/980 [==============================] - 0s 434us/step - loss: 0.5642 - precision_metrics: 0.5901
Epoch 4/150
980/980 [==============================] - 0s 391us/step - loss: 0.5622 - precision_metrics: 0.5918
Epoch 5/150
980/980 [==============================] - 0s 408us/step - loss: 0.5616 - precision_metrics: 0.5918
Epoch 6/150
980/980 [==============================] - 0s 405us/step - loss: 0.5611 - precision_metrics: 0.5918
Epoch 7/150
980/980 [==============================] - 0s 388us/step - loss: 0.5608 - precision_metrics: 0.5918
Epoch 8/150
980/980 [==============================] - 0s 428us/step - loss: 0.5608 - precision_metrics: 0.5918
Epoch 9/150
980/980 [==============================] - 0s 432us/step - loss: 0.5602 - precision_metr

980/980 [==============================] - 0s 452us/step - loss: 0.5464 - precision_metrics: 0.6090
Epoch 73/150
980/980 [==============================] - 0s 395us/step - loss: 0.5455 - precision_metrics: 0.6096
Epoch 74/150
980/980 [==============================] - 0s 412us/step - loss: 0.5454 - precision_metrics: 0.6110
Epoch 75/150
980/980 [==============================] - 0s 414us/step - loss: 0.5456 - precision_metrics: 0.6147
Epoch 76/150
980/980 [==============================] - 0s 411us/step - loss: 0.5456 - precision_metrics: 0.6166
Epoch 77/150
980/980 [==============================] - 0s 381us/step - loss: 0.5454 - precision_metrics: 0.6108
Epoch 78/150
980/980 [==============================] - 0s 416us/step - loss: 0.5447 - precision_metrics: 0.6102
Epoch 79/150
980/980 [==============================] - 0s 398us/step - loss: 0.5455 - precision_metrics: 0.6091
Epoch 80/150
980/980 [==============================] - 0s 423us/step - loss: 0.5449 - precision_metrics: 0.6

980/980 [==============================] - 0s 401us/step - loss: 0.5406 - precision_metrics: 0.6212
Epoch 145/150
980/980 [==============================] - 0s 381us/step - loss: 0.5406 - precision_metrics: 0.6148
Epoch 146/150
980/980 [==============================] - 0s 375us/step - loss: 0.5411 - precision_metrics: 0.6130
Epoch 147/150
980/980 [==============================] - 0s 393us/step - loss: 0.5406 - precision_metrics: 0.6193
Epoch 148/150
980/980 [==============================] - 0s 377us/step - loss: 0.5405 - precision_metrics: 0.6196
Epoch 149/150
980/980 [==============================] - 0s 375us/step - loss: 0.5404 - precision_metrics: 0.6151
Epoch 150/150
980/980 [==============================] - 0s 373us/step - loss: 0.5398 - precision_metrics: 0.6153


In [26]:
print("Train set")
y_pred = model.predict(X_train).squeeze()
y_pred = y_pred.argmax(axis=1)
print(metrics.confusion_matrix(y_train, y_pred))
print(metrics.classification_report(y_train, y_pred))
print(metrics.accuracy_score(y_train, y_pred))
print("Test set")
y_pred = model.predict(X_test).squeeze()
y_pred = y_pred.argmax(axis=1)
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

Train set
[[  5   2 193]
 [  0  13 187]
 [  0   1 579]]
             precision    recall  f1-score   support

          0       1.00      0.03      0.05       200
          1       0.81      0.07      0.12       200
          2       0.60      1.00      0.75       580

avg / total       0.73      0.61      0.48       980

0.6091836734693877
Test set
[[  0   1  39]
 [  0   0  12]
 [  0   0 248]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        40
          1       0.00      0.00      0.00        12
          2       0.83      1.00      0.91       248

avg / total       0.69      0.83      0.75       300

0.8266666666666667


c:\users\siddp\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
